In [28]:
import folium
import pandas as pd
import csv
from datetime import datetime

In [56]:
with open('Building_Permits___Current.csv',encoding='ascii',errors='ignore') as csvfile:
    permits = pd.read_csv(csvfile)
#  Change data types to reflect actual data types   
permits['Issue Date']= pd.to_datetime(permits['Issue Date'], format='%m/%d/%y')
permits['Final Date']= pd.to_datetime(permits['Final Date'], format='%m/%d/%y')
permits['Expiration Date']= pd.to_datetime(permits['Expiration Date'], format='%m/%d/%y')
permits['Application Date']= pd.to_datetime(permits['Application Date'], format='%m/%d/%y')

# Change permit value from type Object to Type float, so we can choose permits above a specified value
permits['Value'] = permits['Value'].map(lambda x: x.lstrip('$'))
permits['Value'] =permits['Value'].str.replace(',', '').astype(float)

In [98]:
core_permits = permits.filter(['Application/Permit Number', 'Address','Description', 'Category','Action Type','Value','Application Date','Issue Date','Final Date','Expiration Date','Status','Latitude','Longitude','Location'])
core_permits = core_permits[core_permits['Final Date'].notnull()]
core_permits = core_permits[core_permits['Action Type'] == 'NEW']
core_permits = core_permits[core_permits['Value'] > 500000]
length = core_permits.shape[0]
length

971

In [99]:
# long_permits = permits[str(permits['Final Date'] != 'NaT')] 
core_permits.dtypes
import numpy as np
core_permits['Accident_Count'] = np.zeros(length)+1

In [85]:
map_osm = folium.Map(location=[47.62118988, -122.2980787], zoom_start = 11)

In [80]:
map_osm 


In [63]:
def add_marker(data, map_object):
    for index, row in data.iterrows():
        folium.CircleMarker(
        location = [row['Latitude'], row['Longitude']],
        radius=row['Accident_Count'],
        fill_color='#132b5e'
        ).add_to(map_object)


In [100]:
map_osm = folium.Map(location=[47.62118988, -122.2980787], zoom_start = 11)
add_marker(core_permits, map_osm)

map_osm